In [1]:
!pip install cvxopt

    100% |████████████████████████████████| 16.1MB 39kB/s eta 0:00:011   10% |███▌                            | 1.8MB 19.0MB/s eta 0:00:01    43% |██████████████                  | 7.1MB 11.0MB/s eta 0:00:01
You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install pystruct

    100% |████████████████████████████████| 5.6MB 206kB/s eta 0:00:01    62% |████████████████████▏           | 3.5MB 11.0MB/s eta 0:00:01
    100% |████████████████████████████████| 2.3MB 335kB/s eta 0:00:01
  Running setup.py bdist_wheel for pystruct ... done
  Stored in directory: /home/nbuser/.cache/pip/wheels/8b/87/bc/6fb10e64e8fd0b722e9e9e2236a939a1e9957d792b7b77486b
Successfully built pystruct
You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.linalg import solve, inv, norm

from cvxopt import matrix, solvers

from pystruct.inference import inference_dispatch

Пусть $y_i \in \{0,1,...,K-1\}$ для $i=1,2$. Найдем решение задачи дискретной оптимизации:

$$ \min_y \theta_1(y_1) + \theta_2(y_2) + \theta_{12}(y_1,y_2)$$

In [4]:
K = 5

theta_s = np.random.randn(K,)
theta_t = np.random.randn(K,)
theta_st = np.random.randn(K,K)

Наивное решение: полный перебор всех $K^2$ конфигураций переменных $y = (y_1, y_2)$.

In [5]:
f_min = np.inf

for i in range(K):
    for j in range(K):
        f = theta_s[i] + theta_t[j] + theta_st[i,j]
        if f < f_min:
            f_min = f
            y_sol = [i, j]
            
print('Solution:', y_sol)

Solution: [4, 2]


LP-релаксация:

\begin{gather}
\underset{\mu}{\text{min}} & \theta_1^T\mu_1 + \theta_2^T\mu_2 + \theta_{12}^T\mu_{12} \\
\text{s.t.} & 1^T\mu_1 = 1, 1^T\mu_2 = 1 \\
& M_{\text{vert}}\mu_{12} = \mu_2, M_{\text{horz}}\mu_{12} = \mu_1 \\
& \mu \geq 0
\end{gather}

Этот метод также реализован в [pystruct.inference.inference_lp](https://pystruct.github.io/generated/pystruct.inference.inference_lp.html) (очень медленный).

In [6]:
I = np.eye(K)
ones = np.ones((1,K))

M_vert = np.kron(I, ones)
M_horz = np.kron(ones, I)

#print(M_vert)
#print(M_horz)

In [7]:
theta = np.hstack((theta_s, theta_t, theta_st.T.flatten())) # transp !

In [8]:
c = theta

O = np.zeros((K,K))
G1 = np.hstack((O, -I, M_vert))
G2 = np.hstack((-I, O, M_horz))
G3 = np.hstack((ones, 0.*ones, np.zeros((1,K**2))))
G4 = np.hstack((0*ones, ones, np.zeros((1,K**2))))
G = np.vstack((G1, G2, G3, G4))
h = np.vstack((np.zeros((2*K,1)), np.ones((2,1))))   

A = -np.eye(K+K+K**2)
b = np.zeros((K+K+K**2,1))

# objective
c = matrix(c)
# inequalities
A = matrix(A)
b = matrix(b)
# equalities
G = matrix(G)
h = matrix(h)


sol=solvers.lp(c, A, b, G, h, solver='glpk')
#print(sol)
xx = np.array(sol['x'])
x1 = xx[:K] # mu_1
x2 = xx[K:2*K] # mu_2

idx1 = np.argmax(x1)
idx2 = np.argmax(x2)

print('Solution:', [idx1, idx2])

Solution: [4, 2]


Динамическое программирование:

Целевая функция соответствует графу, состоящему из 2-х вершин, соединенных ребром.

<img src="http://apprize.info/data/nosql/nosql.files/image147.jpg" width=200>

Граф является _ациклическим_ (дерево), поэтому точное решение можно найти с помощью динамического программирования (в этом случае LP-релаксация также дает точное решение). Исходная задача сводится к поиску пути минимальной стоимости в графе вида (multistage graph):

<img src="https://www.researchgate.net/profile/Bartosz_Musznicki/publication/265412778/figure/fig1/AS:392039154896896@1470480826291/A-general-structure-of-a-multi-stage-graph.png" width=500>

https://pystruct.github.io/generated/pystruct.inference.inference_dispatch.html

In [ ]:
n_nodes = 2
n_edges = 1
n_states = 3

unary_potentials = np.zeros((n_nodes, n_states))
pairwise_potentials = np.zeros((n_edges, n_states, n_states))
edges = np.zeros((n_edges, 2), dtype=np.int32) # int

unary_potentials[0,:] = theta_s
unary_potentials[1,:] = theta_t
pairwise_potentials[0,:,:] = theta_st
edges[0,0] = 0

edges[0,1] = 1

# Find the maximizing assignment of a pairwise discrete energy function
y = inference_dispatch(-1.0*unary_potentials, -1.0*pairwise_potentials, edges, inference_method='max-product')

print('Solution:', y)

Solution: [4 2]
